In [8]:
import pandas as pd
import asyncio
import aiohttp
import time
import json

In [9]:
doktorlar_detay = pd.read_csv('doktorlar_detay.csv')
doktorlar_detay.head()

,id,image_link,title,name,is_verified,profile,gender,location,speciality,data_id,expert_point,services,about_content,eed_items,expert_interests,experience_company
0,1,https://www.doktorsitesi.com/media/cache/profi...,Psk.,Arzu Kantarcıoğlu,NaN,https://www.doktorsitesi.com/psk-arzu-kantarci...,female,bursa,psikoloji,10344.0,NaN,[],NaN,[],[],NaN
1,2,https://www.doktorsitesi.com/media/cache/profi...,Dr.,Abdullah Atasoy,NaN,https://www.doktorsitesi.com/dr-abdullah-ataso...,male,balikesir,aile-hekimligi,10953.0,NaN,[],NaN,[],[],NaN
2,3,https://www.doktorsitesi.com/media/cache/profi...,Dr.,Atakan Saçlı,NaN,https://www.doktorsitesi.com/dr-atakan-sacli/p...,male,izmir,pratisyen-hekimlik,65885.0,NaN,[],NaN,[],"[""Bel F\u0131t\u0131\u011f\u0131"", ""Sch\u00fcs...",NaN
3,4,https://www.doktorsitesi.com/media/cache/profi...,Op. Dr.,Ayla Özer(ağar),NaN,https://www.doktorsitesi.com/op-dr-ayla-ozerag...,female,izmir,kadin-hastaliklari-ve-dogum,180072.0,NaN,[],NaN,"[""Diploma Ald\u0131\u011f\u0131 Kurum\nEGE \u0...","[""Adet A\u011fr\u0131lar\u0131 (Dismenore)"", ""...",NaN
4,5,https://www.doktorsitesi.com/media/cache/profi...,Op. Dr.,Ayşe Gülgün Bilgin,NaN,https://www.doktorsitesi.com/op-dr-ayse-gulgun...,female,mugla,kadin-hastaliklari-ve-dogum,180087.0,NaN,[],NaN,"[""Diploma Ald\u0131\u011f\u0131 Kurum\nEGE \u0...","[""Plasenta Previa"", ""K\u00fcrtaj"", ""Adet D\u00...",NaN


In [10]:
data_id = doktorlar_detay['data_id'].dropna().astype(int).to_list()
len(data_id)

19882

In [14]:


# https://api.doktorsitesi.com/public/question-thread/profile/180089?page=1&limit=100000

# the response is a json object

zero_responses = []
error_data_ids = []

start_time = time.time()
async def fetch(session, url, data_id):
    try:
        async with session.get(url) as response:
            res = await response.json()
            # {"count": "0", "items": []}
            # save the response to a json file with the name of the data_id in the soru_cevap folder
            if res['count'] == '0':
                zero_responses.append(data_id)
                print(f"{url} has no response")
            else:
                with open(f'soru_cevap/{data_id}.json', 'w') as f:
                    json.dump(res, f, ensure_ascii=False)
            
                print(f"{url} is fetched")
    except Exception as e:
        print(f"Error occured while fetching {url}")
        print(e)
        error_data_ids.append(data_id)
async def fetch_all(urls):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i, url in enumerate(urls):
            task = asyncio.create_task(fetch(session, url, data_id[i]))
            tasks.append(task)
        await asyncio.gather(*tasks)
    
urls = [f'https://api.doktorsitesi.com/public/question-thread/profile/{data_id[i]}?page=1&limit=1000000' for i in range(len(data_id))]
await fetch_all(urls)

print("--- %s seconds ---" % (time.time() - start_time))


https://api.doktorsitesi.com/public/question-thread/profile/180664?page=1&limit=1000000 has no response
https://api.doktorsitesi.com/public/question-thread/profile/180948?page=1&limit=1000000 has no response
https://api.doktorsitesi.com/public/question-thread/profile/180302?page=1&limit=1000000 is fetched
https://api.doktorsitesi.com/public/question-thread/profile/180947?page=1&limit=1000000 has no response
https://api.doktorsitesi.com/public/question-thread/profile/180087?page=1&limit=1000000 is fetched
https://api.doktorsitesi.com/public/question-thread/profile/180976?page=1&limit=1000000 has no response
https://api.doktorsitesi.com/public/question-thread/profile/180072?page=1&limit=1000000 is fetched
https://api.doktorsitesi.com/public/question-thread/profile/180100?page=1&limit=1000000 has no response
https://api.doktorsitesi.com/public/question-thread/profile/180255?page=1&limit=1000000 has no response
https://api.doktorsitesi.com/public/question-thread/profile/180561?page=1&limit

In [15]:
len(zero_responses)

18396

In [16]:
len(error_data_ids)

0

In [20]:
# add response count to the doktorlar_detay dataframe and save it to a csv file

for di in data_id:
    try:
        with open(f'soru_cevap/{di}.json') as f:
            res = json.load(f)
            doktorlar_detay.loc[doktorlar_detay['data_id'] == di, 'response_count'] = res['count']
    except:
        doktorlar_detay.loc[doktorlar_detay['data_id'] == di, 'response_count'] = 0

In [24]:
doktorlar_detay.to_csv('doktorlar_detay.csv', index=False)

In [26]:
doktorlar_detay.head()

,id,image_link,title,name,is_verified,profile,gender,location,speciality,data_id,expert_point,services,about_content,eed_items,expert_interests,experience_company,response_count
0,1,https://www.doktorsitesi.com/media/cache/profi...,Psk.,Arzu Kantarcıoğlu,NaN,https://www.doktorsitesi.com/psk-arzu-kantarci...,female,bursa,psikoloji,10344.0,NaN,[],NaN,[],[],NaN,1
1,2,https://www.doktorsitesi.com/media/cache/profi...,Dr.,Abdullah Atasoy,NaN,https://www.doktorsitesi.com/dr-abdullah-ataso...,male,balikesir,aile-hekimligi,10953.0,NaN,[],NaN,[],[],NaN,0
2,3,https://www.doktorsitesi.com/media/cache/profi...,Dr.,Atakan Saçlı,NaN,https://www.doktorsitesi.com/dr-atakan-sacli/p...,male,izmir,pratisyen-hekimlik,65885.0,NaN,[],NaN,[],"[""Bel F\u0131t\u0131\u011f\u0131"", ""Sch\u00fcs...",NaN,0
3,4,https://www.doktorsitesi.com/media/cache/profi...,Op. Dr.,Ayla Özer(ağar),NaN,https://www.doktorsitesi.com/op-dr-ayla-ozerag...,female,izmir,kadin-hastaliklari-ve-dogum,180072.0,NaN,[],NaN,"[""Diploma Ald\u0131\u011f\u0131 Kurum\nEGE \u0...","[""Adet A\u011fr\u0131lar\u0131 (Dismenore)"", ""...",NaN,2
4,5,https://www.doktorsitesi.com/media/cache/profi...,Op. Dr.,Ayşe Gülgün Bilgin,NaN,https://www.doktorsitesi.com/op-dr-ayse-gulgun...,female,mugla,kadin-hastaliklari-ve-dogum,180087.0,NaN,[],NaN,"[""Diploma Ald\u0131\u011f\u0131 Kurum\nEGE \u0...","[""Plasenta Previa"", ""K\u00fcrtaj"", ""Adet D\u00...",NaN,32


In [27]:
# get total response count

total_response_count = 0
for di in data_id:
    try:
        with open(f'soru_cevap/{di}.json') as f:
            res = json.load(f)
            total_response_count += int(res['count'])
    except:
        pass

total_response_count

167852

In [30]:
# convert data_id to int if it is not already int or not nan
doktorlar_detay['data_id'] = doktorlar_detay['data_id'].apply(lambda x: int(x) if not pd.isna(x) else x)

# save it to a csv file
doktorlar_detay.to_csv('doktorlar_detay.csv', index=False)

In [45]:
doktorlar_detay.head()

,id,image_link,title,name,is_verified,profile,gender,location,speciality,data_id,expert_point,services,about_content,eed_items,expert_interests,experience_company,response_count
0,1,https://www.doktorsitesi.com/media/cache/profi...,Psk.,Arzu Kantarcıoğlu,NaN,https://www.doktorsitesi.com/psk-arzu-kantarci...,female,bursa,psikoloji,10344.0,NaN,[],NaN,[],[],NaN,1
1,2,https://www.doktorsitesi.com/media/cache/profi...,Dr.,Abdullah Atasoy,NaN,https://www.doktorsitesi.com/dr-abdullah-ataso...,male,balikesir,aile-hekimligi,10953.0,NaN,[],NaN,[],[],NaN,0
2,3,https://www.doktorsitesi.com/media/cache/profi...,Dr.,Atakan Saçlı,NaN,https://www.doktorsitesi.com/dr-atakan-sacli/p...,male,izmir,pratisyen-hekimlik,65885.0,NaN,[],NaN,[],"[""Bel F\u0131t\u0131\u011f\u0131"", ""Sch\u00fcs...",NaN,0
3,4,https://www.doktorsitesi.com/media/cache/profi...,Op. Dr.,Ayla Özer(ağar),NaN,https://www.doktorsitesi.com/op-dr-ayla-ozerag...,female,izmir,kadin-hastaliklari-ve-dogum,180072.0,NaN,[],NaN,"[""Diploma Ald\u0131\u011f\u0131 Kurum\nEGE \u0...","[""Adet A\u011fr\u0131lar\u0131 (Dismenore)"", ""...",NaN,2
4,5,https://www.doktorsitesi.com/media/cache/profi...,Op. Dr.,Ayşe Gülgün Bilgin,NaN,https://www.doktorsitesi.com/op-dr-ayse-gulgun...,female,mugla,kadin-hastaliklari-ve-dogum,180087.0,NaN,[],NaN,"[""Diploma Ald\u0131\u011f\u0131 Kurum\nEGE \u0...","[""Plasenta Previa"", ""K\u00fcrtaj"", ""Adet D\u00...",NaN,32


## 167.852 responses